In [1]:
# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np


In [2]:
#!pip3 install zeroconf
#!pip3 install requests

# Connect to the microscope 
from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000)


In [20]:
def whole_platescan_96well(microscope, offset_x=2000, offset_y=3000, xstart=0, ystart=0, Nx = 8, Ny = 12, name='TestScan'):

    # perform a scan of the whole well plate
    dx = 9000
    dy = 9000

    # move to the home positoin 
    Nx = Nx-xstart
    Ny = Ny-ystart
    microscope.move((offset_x+dx*xstart,offset_y+dy*ystart,0), True)

    params={'annotations': {'Client':'openflexure-microscope-jsclient:builtin'}, 
     'autofocus_dz': 0, 
     'bayer': False, 
     'fast_autofocus': False, 
     'grid': [Nx, Ny, 1], 
     'namemode': 'number', 
     'stride_size': [dx, dy, 1], 
     'style': 'snake', 
     'tags': [], 
     'filename':name,       
     'temporary': False, 
     'use_video_port': True}

    microscope.scan(params, wait_on_task=True)
    
    # move to the home positoin 
    microscope.move((offset_x+dx*xstart,offset_y+dy*ystart,0), True)


In [4]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.8')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left')


In [5]:
# setup the labware in the trays
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')
plate2 = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='7')
tiprack = protocol.load_labware('opentrons_96_filtertiprack_200ul',location='8')

In [6]:
# homing the robot and get rid of all pipettes
protocol.home()
pipette.home()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [7]:
def pickup_fresh_pipette_tip(i_pipette, pipette, tiprack):
    try:
        my_fresh_tip = tiprack.wells()[i_pipette]
        pipette.pick_up_tip(my_fresh_tip)
    except:
        i_pipette += 1
        print("no pipette attached")
        pipette.drop_tip()
        pipette.pick_up_tip(my_fresh_tip)

    return i_pipette

       
# i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)


In [43]:
#pipette.home()
time_last_scan = -10000
time_last_pipette = -10000
i_experiment = 1
i_pipette = 0 
#i_id = 6

In [44]:
i_id =28

# Do the actual pipetting and imaging

In [ ]:

# 
T_period_scan = 1*60  # take a whole plate scan every 5 minutes
T_period_pipette = 20*60 # add a new reagent every 60 minutes  


N_rows = 4 
N_cols = len(plate.columns()[0])
V_aspirate = 50 # µm
iiter = 0


# set speed
protocol.max_speeds['X'] = 200 # default = 400
protocol.max_speeds['Y'] = 200 # default = 300


# start dispensing 
experiment_methods = ['50-ethanol-50-PBS', 'pbs', 'salt', 'none']
last_exerpiment = 'none'

# microscope parameters
offset_x=3000
offset_y=3000

# Make sure microscope is homed!
# microscope.move((0*9000+offset_x,0*9000+offset_y,0), True)

# Perform a whole plate scan

time_now = time.time()
i_row=-1
while(True):
    # start a full-well scan every 5 minutes
    time_now = time.time()
    if (time_now - time_last_scan) > T_period_scan:
        experiment_name = experiment_methods[i_row]
        whole_platescan_96well(microscope, offset_x=offset_x, offset_y=offset_y, xstart=0, ystart=8, Nx = 8, Ny = 12, name='TestScan'+str(i_id))
        time_last_scan = time.time()
        i_id +=1
        
        # 
        microscope.move((offset_x,offset_y+9000*6,0), True)

    # start a full-well scan every 30 minutes
    if (time_now - time_last_pipette) > T_period_pipette:  
              
        time_last_pipette = time.time()
        for i_row in range(N_rows):

            # select the exerpiment and use the proper pipette tip
            current_exerpiment = experiment_methods[i_row]


            if current_exerpiment is not "none":
                
                # pickup new pipette since we have a new reagent
                i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)

                for i_col in range(i_experiment, N_cols):
                    '''
                    Go to the liquid and aspirate some stuff
                    '''
                    #get position of the first well of the microscope
                    print("Picking up: "+experiment_methods[i_row])
                    pos_reagent = plate2.columns(i_row)[0][0].center()
                    dx=0; dy=-3; dz=-15 # offset-coordinates (only calibrate it once)

                    # Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
                    adjusted_location = pos_reagent.move(types.Point(x=dx, y=dy, z=dz))
                    pipette.move_to((adjusted_location))
                    pipette.aspirate(V_aspirate)

                    # move head up
                    pos_reagent = plate2.rows(i_row)[0][0].center()
                    # Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
                    adjusted_location = pos_reagent.move(types.Point(x=dx, y=dy, z=130))
                    pipette.move_to((adjusted_location))

                    '''
                    Go to the microscope and dispense 
                    '''    
                    #get position of the first well of the microscope
                    center_location = plate.columns()[i_row][i_col].center()
                    # offset-coordinates (only calibrate it once)
                    dx=-6; dy=2; dz=150

                    # Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
                    adjusted_location = center_location.move(types.Point(x=dx, y=dy, z=135))
                    pipette.move_to((adjusted_location))

                    # Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
                    adjusted_location = center_location.move(types.Point(x=dx, y=dy, z=88))
                    pipette.move_to((adjusted_location))
                    pipette.dispense()
                    
                    # fully dispense
                    adjusted_location = center_location.move(types.Point(x=dx, y=dy, z=95))
                    pipette.move_to((adjusted_location))
                    pipette.blow_out()

                    # move up
                    # Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
                    adjusted_location = center_location.move(types.Point(x=dx, y=dy, z=135))
                    pipette.move_to((adjusted_location))


        i_experiment += 1  
                




Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
no pipette attached


Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
no pipette attached


Picking up: salt
Picking up: salt
Picking up: salt
Picking up: salt
Picking up: salt
Picking up: salt
Picking up: salt
no pipette attached


Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
Picking up: 50-ethanol-50-PBS
no pipette attached


Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
Picking up: pbs
no pipette attached


...

no pipette attached


no pipette attached


no pipette attached


no pipette attached


no pipette attached


In [32]:
i_pipette

1

In [ ]:
pipette.drop_tip()

In [ ]:
pipette.home()

In [ ]:
center_location = plate2['A1'].center()

# Get a location 1 mm right, 1 mm back, and 1 mm up from the center of well A1.
adjusted_location = center_location.move(types.Point(x=1, y=1, z=110))

pipette.move_to((adjusted_location))


In [ ]:
pipette.home()